In [ ]:
import numpy as np
from util import generate_sparse_least_squares
from optim import primal_gradient as pg, dual_gradient as dg, nesterov_accelerated_gradient as nag

In [ ]:
def find_magnitude_indices(gap_history, func=np.log2):
    magnitudes = np.floor(func(gap_history))
    unique_magnitudes = []
    indices = []

    for i, magnitude in enumerate(magnitudes):
        if magnitude not in unique_magnitudes:
            unique_magnitudes.append(magnitude)
            indices.append(i)
    indices = np.array(indices) + 1
    return indices

In [184]:
import numpy as np

def primal_gradient_method_with_history(A, b, x0, tol, max_iter=10000):
    """
    Implement the primal gradient method to solve Ax = b, tracking both gap history and loss history.
    
    Args:
        A (np.ndarray): Coefficient matrix.
        b (np.ndarray): Right-hand side vector.
        x0 (np.ndarray): Initial guess for the solution.
        tol (float): Tolerance for stopping criterion based on relative gap.
        max_iter (int): Maximum number of iterations.
    
    Returns:
        Tuple containing:
        - np.ndarray: The solution vector x.
        - int: Total number of iterations performed.
        - float: Final relative gap.
        - list: History of the relative gap during iterations.
        - list: History of the loss (norm of residual squared) during iterations.
    """
    m, n = A.shape
    x = x0
    r = np.dot(A, x) - b  # initial residual
    initial_loss = np.linalg.norm(r)**2
    gap_history = []
    loss_history = [initial_loss]

    for k in range(max_iter):
        gradient = 2 * np.dot(A.T, (np.dot(A, x) - b))
        step_size = 0.1 / np.linalg.norm(gradient)  # simple step size rule
        x -= step_size * gradient
        r = np.dot(A, x) - b
        current_loss = np.linalg.norm(r)**2
        relative_gap = current_loss / initial_loss
        loss_history.append(current_loss)
        gap_history.append(relative_gap)
        
        if relative_gap < tol:
            break

    return x, k, relative_gap, gap_history, loss_history

# Parameters
m, n = 1000, 4000  # dimensions
rho = 0.5          # sparsity factor
tol = 1e-9         # tolerance for relative gap reduction

# Generate the problem
A, b, x_star = generate_sparse_least_squares(m, n, rho)

# Initial guess
x0 = np.zeros(n)

# Solve using primal gradient method with history tracking
solution, iterations, final_gap, gap_history, loss_history = primal_gradient_method_with_history(A, b, x0, tol)

print("Solution found after {} iterations with final relative gap: {:.4f}".format(iterations, final_gap))
print("Gap history:", gap_history)
print("Loss history:", loss_history)


Solution found after 9999 iterations with final relative gap: 0.0000
Gap history: [0.9899087951365564, 0.9798764843695874, 0.9699030001923395, 0.9599882746265629, 0.9501322392176299, 0.9403348250295899, 0.9305959626401633, 0.9209155821356674, 0.9112936131058811, 0.9017299846388408, 0.8922246253155692, 0.8827774632047385, 0.8733884258572627, 0.864057440300821, 0.8547844330343087, 0.8455693300222212, 0.8364120566889586, 0.827312537913061, 0.8182706980213702, 0.8092864607831098, 0.8003597494038961, 0.7914904865196658, 0.7826785941905274, 0.7739239938945317, 0.7652266065213628, 0.7565863523659458, 0.7480031511219729, 0.7394769218753439, 0.7310075830975233, 0.7225950526388097, 0.7142392477215189, 0.7059400849330775, 0.6976974802190298, 0.6895113488759494, 0.6813816055442641, 0.6733081642009836, 0.6652909381523371, 0.6573298400263123, 0.6494247817651007, 0.6415756746174449, 0.6337824291308867, 0.6260449551439157, 0.6183631617780178, 0.610736957429623, 0.603166249761946, 0.5956509456967284, 0

In [187]:
find_magnitude_indices(gap_history, func=np.log2)

array([  1,  60, 105, 139, 165, 185, 200, 212, 220, 227, 231, 235, 237,
       239, 240, 241, 242, 243, 245, 247, 249, 250])

In [168]:
import numpy as np

def generate_sparse_least_squares(m, n, rho):
    """ Generate a sparse least squares problem """
    A = np.random.uniform(-1, 1, (m, n))
    x_star = np.zeros(n)
    non_zero_indices = np.random.choice(n, int(n * rho), replace=False)
    x_star[non_zero_indices] = np.random.normal(0, 1, int(n * rho))
    noise = np.random.normal(0, 0.1, m)
    b = np.dot(A, x_star) + noise
    return A, b, x_star

def dual_gradient_method(A, b, v0, L0, gamma_d, max_iter=10000, tol=1e-6):
    """
    Dual Gradient Method adapted for solving Ax = b using least squares
    """
    m, n = A.shape
    v = v0
    L = L0
    k = 0
    initial_residual = np.dot(A, v) - b
    initial_gap = np.linalg.norm(initial_residual)**2
    
    while k < max_iter:
        grad_f = 2 * np.dot(A.T, (np.dot(A, v) - b))  # Gradient of the least squares loss
        Mk = np.linalg.norm(grad_f, 2)                # Computing Mk
        
        if Mk == 0:
            Mk = 1e-16  # Prevent division by zero
        
        # Update step
        v -= grad_f / Mk
        
        # Calculate current residual and gap
        current_residual = np.dot(A, v) - b
        current_gap = np.linalg.norm(current_residual)**2
        relative_gap = current_gap / initial_gap
        
        # Check for convergence
        if relative_gap < tol:
            break
        
        k += 1

    return v, k, relative_gap

# Parameters
m, n = 1000, 4000  # dimensions
rho = 0.5          # sparsity factor
tol = 1e-9         # tolerance for relative gap reduction

# Generate the problem
A, b, x_star = generate_sparse_least_squares(m, n, rho)

# Initial guess
v0 = np.zeros(n)

# Solve using Dual Gradient Method
solution, iterations, final_gap = dual_gradient_method(A, b, v0, 2, 2, tol=tol)

print("Solution found after {} iterations with final relative gap: {:.4f}".format(iterations, final_gap))


Solution found after 10000 iterations with final relative gap: 0.0009


In [178]:
import numpy as np

def dual_gradient_method_with_nesterov_and_history(A, b, v0, L0, gamma_d, max_iter=10000, tol=1e-6):
    """
    Dual Gradient Method with Nesterov acceleration for solving Ax = b using least squares,
    including gap history and loss history.
    """
    m, n = A.shape
    v = v0
    v_prev = v0
    L = L0
    gap_history = []
    loss_history = []
    initial_residual = np.dot(A, v) - b
    initial_loss = np.linalg.norm(initial_residual)**2 / 2

    for k in range(max_iter):
        # Nesterov acceleration step
        if k > 0:
            y = v + (k - 1) / (k + 2) * (v - v_prev)
        else:
            y = v

        grad_f = 2 * np.dot(A.T, (np.dot(A, y) - b))  # Gradient of the least squares loss

        Mk = np.linalg.norm(grad_f, 2)                # Computing Mk
        if Mk == 0:
            Mk = 1e-16  # Prevent division by zero

        v_prev = v
        v -= gamma_d * grad_f / Mk

        current_residual = np.dot(A, v) - b
        current_loss = np.linalg.norm(current_residual)**2 / 2
        current_gap = np.linalg.norm(current_residual)**2 / np.linalg.norm(initial_residual)**2

        gap_history.append(current_gap)
        loss_history.append(current_loss)

        # Check for convergence based on the relative gap
        if current_gap < tol:
            break

    return v, k, current_gap, gap_history, loss_history

# Parameters
m, n = 1000, 4000  # dimensions
rho = 0.5          # sparsity factor
tol = 1e-9         # tolerance for relative gap reduction

# Generate the problem
A, b, x_star = generate_sparse_least_squares(m, n, rho)

# Initial guess and parameters for the method
v0 = np.zeros(n)
L0 = 2
gamma_d = 0.1

# Solve using Dual Gradient Method with enhancements
solution, iterations, final_gap, gap_history, loss_history = dual_gradient_method_with_nesterov_and_history(A, b, v0, L0, gamma_d, tol=tol)

print("Solution found after {} iterations with final relative gap: {:.4f}".format(iterations, final_gap))
print("Gap history:", gap_history)
print("Loss history:", loss_history)


Solution found after 9999 iterations with final relative gap: 0.0000
Gap history: [0.9901401973005565, 0.9803366479727496, 0.970589291075566, 0.9608980652304308, 0.9512629086165156, 0.9416837589659863, 0.932160553559187, 0.9226932292197567, 0.9132817223096835, 0.9039259687242908, 0.8946259038871551, 0.8853814627449577, 0.8761925797622651, 0.8670591889162397, 0.8579812236912776, 0.8489586170735757, 0.8399913015456242, 0.8310792090806244, 0.8222222711368306, 0.8134204186518149, 0.8046735820366561, 0.7959816911700455, 0.787344675392316, 0.7787624634993886, 0.7702349837366357, 0.7617621637926613, 0.7533439307929973, 0.7449802112937114, 0.7366709312749294, 0.7284160161342694, 0.7202153906801844, 0.7120689791252156, 0.7039767050791532, 0.6959384915421031, 0.6879542608974589, 0.6800239349047804, 0.6721474346925698, 0.6643246807509551, 0.6565555929242691, 0.6488400904035307, 0.6411780917188211, 0.6335695147315581, 0.6260142766266645, 0.6185122939046307, 0.6110634823734694, 0.6036677571405618, 

In [177]:
final_gap

1.4346651108469044e-05

In [182]:
len(find_magnitude_indices(gap_history, func=np.log2))

22